# Data

[150k Python Dataset](https://eth-sri.github.io/py150) from SRILAB.

# Code representation

A continuous vector embedding of each code fragment at method–level granularity as "document".[13] FastText, a variation of Word2Vec algorithm.
- Extracting Information from Source Code
    - simple tokenizer: extract all words from source code by removing non–alphanumeric tokens.=> indifferenciable
    - parser-based approach: traverse through the parse tree for each method, and extract information from the following syntactic categories. (Java-like)
        - method name
        - method invocation
        - Enums
        - String literals
        - comments
        - <strike>variable name</strike>
- Building Vector Representations
    - <strike>simply average embeddings</strike>
    - Weighted average of all unique words in a document=> normalized tf-idf
- Retrieval
    - average the vector representations of constituent words to create a document embedding for the query sentence
    - a standard similarity search algorithm to find the document vectors with closest cosine distance. => FAISS 

# Model

Input: natural language queries <br>
Output: related code fragments retrieved directly from Github code corpus<br><br>

Map the Input into the same vector space as the codebase, and then calculate the vector distance of them in order to get the relevant result.

# Evaluation

Metric and choosing parameters of the model

- Metric: select subsets of words from the document as simulated queries and then see if it can retrive the document, and then evaluate by the percentage of the documents that are retrieve back at top1 and top10. 
    - random benchmark test
    - TF-IDF benchmark test => better performance
- Parameters:
    - embedding dimention=> 500
    - three ways of combining word embeddings to document embeddings=> the conclusion is tf-idf better
    - vector representation=> BM25 is better

# Implementation

In [17]:
import numpy as np
import pandas as pd
import string
import re
import pickle

from gensim.models import FastText
from gensim.models import KeyedVectors
from time import time

In [18]:
# change parameters here
function_file_path="pickle100.pkl"
wordembedding_file_path="data/embeddings.txt"
docembedding_file_path="data/document_embeddings.csv"

## Data Processing

In [19]:
# load in processed file. A set of keywords for each document (source code function)
def load_words_from_ast(file_path):
    with open(file_path, 'rb') as f:
        function_list = pickle.load(f)
    unpickled_df = pd.DataFrame(function_list, columns=['data_id', 'function_name', 'docstring', 'func_call'])

    func_size=len(unpickled_df)
    print("Total Number of Functions in \"{}\": {}".format(file_path, func_size))
    return unpickled_df

unpickled_df=load_words_from_ast(function_file_path)

Total Number of Functions in "pickle100.pkl": 1038


In [20]:
# substract useless function from code base 
# 1 - keep, 0 - delete from code base
unpickled_df['keep_in_codebase'] = np.where(((unpickled_df.function_name == unpickled_df.func_call)|
                                             (unpickled_df['func_call'] == '')|
                                            (unpickled_df['function_name'] == '__init__')
                                            ), 0, 1)
unpickled_df.head()

,data_id,function_name,docstring,func_call,keep_in_codebase
0,1,__getattribute__,,__getattribute__,0
1,1,__setattr__,,"ref,__setattr__",1
2,2,main,,"setup,closing,ZmqProxy,consume_in_thread,wait",1
3,3,test_vpnservice_create,,"create_stubs,first,AndReturn,ReplayAll,vpnserv...",1
4,3,test_vpnservices_get,,"create_stubs,AndReturn,ReplayAll,vpnservices_g...",1


In [3]:
def parse_func_name(terms):
    """
    Return a list of keywords from function name
    """
    result=[]
    #1. alphanumeric and "-", "_" only
    terms=re.sub("[^\w_-]", " ", terms)
    
    #2. snake case
    result=[term for term in terms.split("_") if term!=""]
    
    #3. camel case
    ## https://stackoverflow.com/questions/29916065/how-to-do-camelcase-split-in-python
    def camel_case_split(identifier):
        matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
        return [m.group(0) for m in matches]
    
    result=[camel_case_split(t) for t in result]
    
    #flatten the result
    #[TODO] check output
    result=[s if isinstance(t, list) else t for t in result for s in t] 
    result=set(result)
    # [TODO] deal with __init__

    # lowercase
    result=[t.lower() for t in result]
    
    return result

def parse_docstring(df):
    #[TODO]remove punctuations except for "_"
    ## reference: https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
    terms=df
    regex = re.compile('[%s]' % re.escape(string.punctuation.replace("_", "")))
    terms=regex.sub(' ', terms)
    return terms
    
def parse_func_call(terms):
    result=[]
    terms_list=terms.split(",")
    for term in terms_list:
        result+=parse_func_name(term)
    return result

In [4]:
## Parsing examples
##parse_docstring("test 123, real_test.")
#parse_func_name("UnpickledDfTest")
#parse_func_call("setup,closing,ZmqProxy,consume_in_thread,wait")

In [5]:
#[TODO] wrap processing into pipeline
#def parse_keywords(df):
#    data = (
#        data
#        # Clean Data
#        .pipe(parse_func_name)
#        .pipe(fix_missing_desc)
#        .pipe(change_lowercase_title)
#        .pipe(change_lowercase_desc)
#        
#        # Transform data
#        .pipe(select_columns, 
#              "item_id",
#              "title",
#              "description",
#              "deal_probability"
#             )
#        .pipe(add_char_len_title)
#        .pipe(add_char_len_desc)
#        .pipe(add_tfidf_title)
#        .pipe(add_tfidf_desc)
#        #.pipe(add_keywords_desc)
#    ) 

In [6]:
# for each function, combine all the keywords into a set
def parse_keywords(df):
    list_function_keywords=[]
    for idx in range(len(df)):
        keywords=[]

        func_name=parse_func_name(unpickled_df.iloc[idx]["function_name"])
        keywords+=func_name
        
        # [TODO] exact possible function names (snakecase or camelcase)
        if unpickled_df.iloc[idx]["docstring"]:
            docstring=unpickled_df.iloc[idx]["docstring"].lower().split()
            keywords+=docstring

        if unpickled_df.iloc[idx]["func_call"]:
            func_invoc=parse_func_call(unpickled_df.iloc[idx]["func_call"])
            keywords+=func_invoc

        list_function_keywords.append(set(keywords))
    return list_function_keywords

list_function_keywords=parse_keywords(unpickled_df)

In [7]:
len(list_function_keywords) #742490 for 100k

1038

## Building Word Embeddings

In [8]:
# hyperparameters
#num_list_function_keywords=742490
vocab_size=500
window_size=5
min_count=1


# other parameters defined earlier
func_size=len(unpickled_df)

In [9]:
# We employ the continuous skip–gram model with a window size of 5, 
# i.e. all pairs of words within distance 5 are considered nearby words.
st=time()
#[TODO] tuning hyperparameters
model = FastText(size=vocab_size, window=window_size, min_count=min_count)  # instantiate
model.build_vocab(sentences=list_function_keywords)
model.train(sentences=list_function_keywords, total_examples=len(list_function_keywords), epochs=10)  # train
print("Run time: {} s".format(time()-st))

In [10]:
print(model)

FastText(vocab=2187, size=200, alpha=0.025)


In [11]:
# saving a model trained via Gensim's fastText implementation
# 2019/03/05: the model might be too big. Saving word vector only.
# model.save('saved_model_gensim')

In [12]:
#[TODO] normalize word mebedding
trained_ft_vectors = model.wv
# save vectors to file if you want to use them later
trained_ft_vectors.save_word2vec_format(wordembedding_file_path, binary=False)

In [13]:
# load wordembedding
st=time()
trained_ft_vectors = KeyedVectors.load_word2vec_format(wordembedding_file_path)
print("Run time: {} s".format(time()-st))

Run time: 0.4404568672180176 s


In [14]:
# Test
trained_ft_vectors.most_similar("pop", topn=10)

[('popitem', 0.9999849796295166),
 ('popitemlist', 0.9999837875366211),
 ('setitem', 0.9999821186065674),
 ('listvalues', 0.9999821186065674),
 ('version', 0.9999816417694092),
 ('versionadded::', 0.9999816417694092),
 ('formatted', 0.9999815225601196),
 ('serverprofiledescription:', 0.9999815225601196),
 ('authentication', 0.9999814629554749),
 ('additional', 0.9999814629554749)]

## Building Document Embeddings

1. Average over all the words;
2. Average over the unique words in each document;
3. [x] Weighted average of all unique words in a document

In [15]:
trained_ft_vectors["ping"]

array([-3.48919451e-01,  1.51616216e-01,  1.49171427e-01, -1.45943258e-02,
       -1.04194053e-01,  1.08259462e-01, -2.35674560e-01, -1.81718558e-01,
       -2.63199985e-01,  2.06316218e-01, -1.80629060e-01,  2.53625661e-01,
        2.92890608e-01, -5.10851681e-01,  2.78451264e-01,  7.92294323e-01,
       -6.45732701e-01,  5.83183020e-02,  4.31636237e-02,  3.10843915e-01,
        3.76761168e-01, -6.06286786e-02,  1.57849446e-01,  3.57999086e-01,
       -4.04884905e-01,  2.36027073e-02, -7.58397877e-01,  3.18058223e-01,
        1.97602212e-01,  9.15620625e-02,  2.65317738e-01, -1.63785771e-01,
        1.16490103e-01, -1.65787116e-02,  1.83883920e-01, -4.76675510e-01,
       -1.61798194e-03, -1.06274605e-01, -2.77244896e-01, -4.73161936e-02,
        9.17183608e-02, -1.28844991e-01, -3.23222250e-01,  6.52275860e-01,
        1.68597445e-01, -4.19973016e-01, -1.26212448e-01,  4.10499841e-01,
       -1.86344922e-01,  3.78558725e-01, -2.56261110e-01, -2.56293386e-01,
       -1.06969662e-01, -

In [16]:
st=time()
document_embeddings=np.zeros((func_size, vocab_size))
for idx, doc in enumerate(list_function_keywords):
    doc_vec_sum=np.zeros(vocab_size)
    
    for term in doc:
        doc_vec_sum+=trained_ft_vectors[term]
    
    document_embeddings[idx]=doc_vec_sum/len(doc)
    
# save the whole document_embeddings
np.savetxt(docembedding_file_path, document_embeddings, delimiter=",")
print("Run time: {} s".format(time()-st))

In [17]:
document_embeddings[0]

array([-0.21107592,  0.0904646 ,  0.08900643, -0.01055287, -0.0630949 ,
        0.06544897, -0.14075872, -0.10933508, -0.15926448,  0.12434152,
       -0.10959296,  0.15216427,  0.17697507, -0.31016997,  0.16759922,
        0.47700399, -0.38974881,  0.03598445,  0.02537935,  0.1873029 ,
        0.22739637, -0.03653921,  0.09591951,  0.2150961 , -0.24436121,
        0.01391554, -0.45711553,  0.19186637,  0.11925419,  0.05474581,
        0.16074289, -0.09895544,  0.07116915, -0.01077115,  0.11260857,
       -0.28771931, -0.00127529, -0.06331161, -0.16753463, -0.02845665,
        0.0566458 , -0.07940213, -0.19468884,  0.39402261,  0.10224196,
       -0.25256968, -0.07602124,  0.24841116, -0.11205653,  0.22795437,
       -0.15530488, -0.15433094, -0.06563385, -0.1853777 , -0.09565699,
       -0.17379734,  0.4083873 , -0.13816503,  0.04775885, -0.19418085,
       -0.22508293,  0.05323267, -0.1114646 , -0.22722116,  0.03826578,
        0.31120721, -0.1502009 ,  0.03923115,  0.01323341, -0.10

In [18]:
print("{} documents with {} dimentions".format(document_embeddings.shape[0], document_embeddings.shape[1]))

1038 documents with 200 dimentions


## Evaluate Model

In [19]:
trained_ft_vectors["pop"]

array([-2.7597290e-01,  1.1983531e-01,  1.1743154e-01, -1.1983320e-02,
       -8.5660860e-02,  8.4647797e-02, -1.8512435e-01, -1.4348288e-01,
       -2.0916961e-01,  1.6426457e-01, -1.4522627e-01,  2.0132948e-01,
        2.3230293e-01, -4.0663308e-01,  2.1976487e-01,  6.2916785e-01,
       -5.1324308e-01,  4.7161989e-02,  3.5263758e-02,  2.4826415e-01,
        3.0012441e-01, -4.6042144e-02,  1.2703046e-01,  2.8364220e-01,
       -3.2003599e-01,  1.8834826e-02, -6.0459703e-01,  2.5427550e-01,
        1.5728919e-01,  7.2336636e-02,  2.1079786e-01, -1.2925409e-01,
        9.4411172e-02, -1.2877812e-02,  1.4792378e-01, -3.7730339e-01,
       -4.1531859e-04, -8.5618414e-02, -2.1905908e-01, -3.5493601e-02,
        7.5856030e-02, -1.0389531e-01, -2.5891450e-01,  5.1983297e-01,
        1.3484707e-01, -3.3604035e-01, -1.0199998e-01,  3.2755610e-01,
       -1.4815851e-01,  3.0138874e-01, -2.0377013e-01, -2.0326264e-01,
       -8.3405532e-02, -2.4334142e-01, -1.2641591e-01, -2.2769101e-01,
      

# Notes

# Questions
The paper mentions 2 evaluation approach: 1 uses Github only, the other one uses both GitHub and StackOverflow. I'm guessing the former one is for tuning in the development stage; while the later is the final evaluation for the completed system (NCS).